# * VINSIGHT : Subscriber

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Import Transaction

In [2]:
''' Execute transaction '''


# Input parameter
v_start_date = 20240101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B1S000500' --Prepaid Usage Subs
        , 'TB1S000500' --Prepaid Usage Subs : TMH
        , 'DB1S000500' --Prepaid Usage Subs : DTAC
        , 'B1S000600' --Prepaid Active Caller 30D Rolling
        , 'TB1S000600' --Prepaid Active Caller 30D Rolling : TMH
        , 'DB1S000600' --Prepaid Active Caller 30D Rolling : DTAC
        , 'B1S000702' --Prepaid SubBase
        , 'DB1S000702' --Prepaid SubBase : DTAC
        , 'TB1S000702' --Prepaid SubBase : TMH
        , 'B1S000700' --Prepaid Reported SubBase
        , 'DB1S000700' --Prepaid Reported SubBase : DTAC
        , 'TB1S000700' --Prepaid Reported SubBase : TMH
        
        , 'B2S010602' --Postpaid SubBase B2C
        , 'DB2S010602' --Postpaid SubBase B2C : DTAC
        , 'TB2S010602' --Postpaid SubBase B2C : TMH
        , 'B2S010600' --Postpaid Reported SubBase B2C
        , 'DB2S010600' --Postpaid Reported SubBase B2C : DTAC
        , 'TB2S010600' --Postpaid Reported SubBase B2C : TMH
        
        , 'TB3S020604' --FTTx SubBase
        , 'TB3S000600' --FTTx Reported SubBase

        , 'TB4S000500' --TVS Active Subs
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    --AND TM_KEY_DAY IN (20240131, 20240229, 20240331, 20240430, 20240531, 20240630, 20240731, 20240831, 20240930, 20241031, 20241130, 20241231, 20250131, 20250228, 20250331) -->> Year 2024
    --AND TM_KEY_DAY IN (20250131, 20250228, 20250331, 20250430, 20250531) -->> Year 2025
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
    --ORDER BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD
""")


try:
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    print(f'\nDataFrame: {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
    
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20240101

Data as of 2025-06-04, 11:07:43

DataFrame: 8839 rows, 13 columns


In [3]:
''' Automate Currently Period '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
prev_mth = str(chk_src_df['TM_KEY_MTH'].drop_duplicates().astype(int).shift().max())

## Daily

In [4]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

# Check Result
print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


In [5]:
''' Subscriber Daily '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    , 'B2S010600' #Postpaid Reported SubBase B2C
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB3S000600' #FTTx Reported SubBase
	, 'TB4S000500' #TVS Active Subs
    ]

sub_daily_df = chk_src_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'P']].copy()
sub_daily_df = sub_daily_df.loc[sub_daily_df['METRIC_CD'].isin(v_metric_list)]
sub_daily_df = sub_daily_df.loc[sub_daily_df['TM_KEY_MTH']>=v_tm_key_mth]

sub_daily_df['PRE_CALL_30D'] = np.where(sub_daily_df['METRIC_CD']=='B1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_T'] = np.where(sub_daily_df['METRIC_CD']=='TB1S000600', sub_daily_df['P'], 0)
sub_daily_df['PRE_CALL_30D_D'] = np.where(sub_daily_df['METRIC_CD']=='DB1S000600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C'] = np.where(sub_daily_df['METRIC_CD']=='B2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_T'] = np.where(sub_daily_df['METRIC_CD']=='TB2S010600', sub_daily_df['P'], 0)
sub_daily_df['POST_RPT_SUB_B2C_D'] = np.where(sub_daily_df['METRIC_CD']=='DB2S010600', sub_daily_df['P'], 0)
sub_daily_df['TOL_RPT_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB3S000600', sub_daily_df['P'], 0)
sub_daily_df['TVS_ACTV_SUB'] = np.where(sub_daily_df['METRIC_CD']=='TB4S000500', sub_daily_df['P'], 0)

sub_daily_df = sub_daily_df.groupby(['TM_KEY_MTH', 'TM_KEY_DAY']).agg({'PPN_TM':'max', 'PRE_CALL_30D':'sum', 'PRE_CALL_30D_T':'sum', 'PRE_CALL_30D_D':'sum', 'POST_RPT_SUB_B2C':'sum', 'POST_RPT_SUB_B2C_T':'sum', 'POST_RPT_SUB_B2C_D':'sum', 'TOL_RPT_SUB':'sum', 'TVS_ACTV_SUB':'sum'})
sub_daily_df = sub_daily_df.fillna(0).sort_values(by=['TM_KEY_DAY']).reset_index()
sub_daily_df = sub_daily_df[['TM_KEY_MTH', 'TM_KEY_DAY', 'PPN_TM', 'PRE_CALL_30D', 'PRE_CALL_30D_T', 'PRE_CALL_30D_D', 'POST_RPT_SUB_B2C', 'POST_RPT_SUB_B2C_T', 'POST_RPT_SUB_B2C_D', 'TOL_RPT_SUB', 'TVS_ACTV_SUB']]

mod_col_list = sub_daily_df.iloc[:, 3:].columns.tolist()
for col in mod_col_list:
    sub_daily_df[col] = sub_daily_df[col].apply(lambda x: format(x, ',.0f'))
sub_daily_df

,TM_KEY_MTH,TM_KEY_DAY,PPN_TM,PRE_CALL_30D,PRE_CALL_30D_T,PRE_CALL_30D_D,POST_RPT_SUB_B2C,POST_RPT_SUB_B2C_T,POST_RPT_SUB_B2C_D,TOL_RPT_SUB,TVS_ACTV_SUB
0,202505,20250501,2025-06-04 02:32:37,"26,568,494","16,730,396","9,838,098","14,176,206","9,192,981","4,983,225","3,191,841","1,141,064"
1,202505,20250502,2025-06-04 02:32:37,"26,649,375","16,798,106","9,851,269","14,177,511","9,192,969","4,984,542","3,193,849","1,140,807"
2,202505,20250503,2025-06-04 02:32:37,"26,640,242","16,796,420","9,843,822","14,179,642","9,193,535","4,986,107","3,187,160","1,140,581"
3,202505,20250504,2025-06-04 02:32:37,"26,621,620","16,788,470","9,833,150","14,097,392","9,110,307","4,987,085","3,188,949","1,140,439"
4,202505,20250505,2025-06-04 02:32:37,"26,604,806","16,781,086","9,823,720","14,178,032","9,196,396","4,981,636","3,190,866","1,140,114"
5,202505,20250506,2025-06-04 02:32:37,"26,604,015","16,775,944","9,828,071","14,179,049","9,196,451","4,982,598","3,187,559","1,139,671"
6,202505,20250507,2025-06-04 02:32:37,"26,587,879","16,768,764","9,819,115","14,179,782","9,197,271","4,982,511","3,190,183","1,139,671"
7,202505,20250508,2025-06-04 02:32:37,"26,568,787","16,760,144","9,808,643","14,181,763","9,197,790","4,983,973","3,192,380","1,138,644"
8,202505,20250509,2025-06-04 02:32:37,"26,608,313","16,785,995","9,822,318","14,177,915","9,199,288","4,978,627","3,192,380","1,138,290"
9,202505,20250510,2025-06-04 02:32:37,"26,612,607","16,783,795","9,828,812","14,180,504","9,200,074","4,980,430","3,192,380","1,137,136"


## Prep Monthly Data

In [6]:
''' Monthly Summary '''

monthly_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==chk_src_df.groupby(['TM_KEY_MTH', 'METRIC_CD'])['ACTUAL_AS_OF'].transform('max')].copy() # ACTUAL_AGG_MTH
monthly_df = monthly_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'PPN_TM', 'ACTUAL_AS_OF']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum'})
monthly_df = monthly_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
monthly_df['DIFF (C-P)'] = monthly_df['C'] - monthly_df['P']

monthly_df_display = monthly_df.copy()
monthly_df_display['ACTUAL_AS_OF'] = monthly_df_display['ACTUAL_AS_OF'].astype(int)
mod_col_list = monthly_df_display.iloc[:, 6:].columns.tolist()
for col in mod_col_list:
    monthly_df_display[col] = monthly_df_display[col].apply(lambda x: format(x, ',.0f'))
# monthly_df_display

In [7]:
# ''' Sample Data '''

# sample_mth_df = monthly_df_display.loc[monthly_df_display['TM_KEY_MTH']==202501]
# sample_mth_df

# # sample_day_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
# # sample_day_df.tail(3)

## Products Summary

In [8]:
''' Parameter '''

# v_tm_key_mth = curr_mth
v_tm_key_mth = 202505

# Check Result
print(f'v_tm_key_mth: {v_tm_key_mth}')

v_tm_key_mth: 202505


### Prepaid

In [9]:
''' Prepaid Usage Subs '''

v_metric_list = [
	'B1S000500' #Prepaid Usage Subs
    , 'TB1S000500' #Prepaid Usage Subs : TMH
    , 'DB1S000500' #Prepaid Usage Subs : DTAC
    ]

prepaid_usage_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_usage_sub_df = prepaid_usage_sub_df.loc[prepaid_usage_sub_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_usage_sub_df = prepaid_usage_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_usage_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1S000500,Prepaid Usage Subs,2025-06-04 02:32:37,20250529,"28,348,526","28,249,805","28,249,805","28,238,012","28,238,012","98,721"
1,202505,Prepaid,DB1S000500,Prepaid Usage Subs : DTAC,2025-06-04 02:32:37,20250529,"11,347,209","11,347,196","11,347,196","11,335,403","11,335,403",13
2,202505,Prepaid,TB1S000500,Prepaid Usage Subs : TMH,2025-06-04 02:32:37,20250531,"16,885,614","16,787,479","16,787,479","16,787,479","16,787,479","98,135"


In [10]:
''' Prepaid Active Caller 30D Rolling '''

v_metric_list = [
    'B1S000600' #Prepaid Active Caller 30D Rolling
    , 'TB1S000600' #Prepaid Active Caller 30D Rolling : TMH
    , 'DB1S000600' #Prepaid Active Caller 30D Rolling : DTAC
    ]

prepaid_call_30d_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_call_30d_df = prepaid_call_30d_df.loc[prepaid_call_30d_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_call_30d_df = prepaid_call_30d_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_call_30d_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,B1S000600,Prepaid Active Caller 30D Rolling,2025-06-04 02:32:37,20250531,"26,496,116","26,496,116","26,462,294","26,462,186","26,462,186",0
1,202505,Prepaid,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,2025-06-04 02:32:37,20250531,"9,906,645","9,906,645","9,905,215","9,905,215","9,905,215",0
2,202505,Prepaid,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,2025-06-04 02:32:37,20250531,"16,589,471","16,589,471","16,557,079","16,556,971","16,556,971",0


In [11]:
''' Prepaid SubBase '''

v_metric_list = [
    'B1S000702' #Prepaid SubBase
    , 'TB1S000702' #Prepaid SubBase : TMH
    , 'DB1S000702' #Prepaid SubBase : DTAC
    ]

prepaid_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_subbase_df = prepaid_subbase_df.loc[prepaid_subbase_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_subbase_df = prepaid_subbase_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,TB1S000702,Prepaid SubBase : TMH,2025-06-04 02:32:37,20250531,0,"23,721,160","23,561,656","23,560,980","23,560,980","-23,721,160"


In [12]:
''' Prepaid Reported SubBase '''

v_metric_list = [
    'B1S000700' #Prepaid Reported SubBase
    , 'DB1S000700' #Prepaid Reported SubBase : DTAC
    , 'TB1S000700' #Prepaid Reported SubBase : TMH
    ]

prepaid_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
prepaid_report_sub_df = prepaid_report_sub_df.loc[prepaid_report_sub_df['TM_KEY_MTH']==v_tm_key_mth]
prepaid_report_sub_df = prepaid_report_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
prepaid_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Prepaid,TB1S000700,Prepaid Reported SubBase : TMH,2025-06-04 02:32:37,20250531,0,"20,108,592","19,961,598","19,961,598","19,961,598","-20,108,592"


### Postpaid

In [13]:
''' Postpaid SubBase B2C '''

v_metric_list = [
    'B2S010602' #Postpaid SubBase B2C
    , 'DB2S010602' #Postpaid SubBase B2C : DTAC
    , 'TB2S010602' #Postpaid SubBase B2C : TMH
    ]

postpaid_subbase_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.loc[postpaid_subbase_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_subbase_b2c_df = postpaid_subbase_b2c_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
postpaid_subbase_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2S010602,Postpaid SubBase B2C,2025-06-04 02:32:37,20250525,0,"14,296,786","9,197,096","9,197,085","9,197,085","-14,296,786"
1,202505,Postpaid,DB2S010602,Postpaid SubBase B2C : DTAC,2025-06-03 23:18:32,20250529,0,"5,090,473",0,0,0,"-5,090,473"
2,202505,Postpaid,TB2S010602,Postpaid SubBase B2C : TMH,2025-06-04 02:32:37,20250530,0,"9,196,048","9,190,968","9,190,958","9,190,958","-9,196,048"


In [14]:
''' Postpaid Reported SubBase B2C '''

v_metric_list = [
    'B2S010600' #Postpaid Reported SubBase B2C
    , 'DB2S010600' #Postpaid Reported SubBase B2C : DTAC
    , 'TB2S010600' #Postpaid Reported SubBase B2C : TMH
    ]

postpaid_report_sub_b2c_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.loc[postpaid_report_sub_b2c_df['TM_KEY_MTH']==v_tm_key_mth]
postpaid_report_sub_b2c_df = postpaid_report_sub_b2c_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
postpaid_report_sub_b2c_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,Postpaid,B2S010600,Postpaid Reported SubBase B2C,2025-06-04 02:32:37,20250530,0,"14,065,089","14,054,416","14,054,406","14,054,406","-14,065,089"
1,202505,Postpaid,DB2S010600,Postpaid Reported SubBase B2C : DTAC,2025-06-04 02:32:37,20250531,0,"4,966,080","4,960,485","4,960,485","4,960,485","-4,966,080"
2,202505,Postpaid,TB2S010600,Postpaid Reported SubBase B2C : TMH,2025-06-04 02:32:37,20250530,0,"9,101,218","9,096,140","9,096,130","9,096,130","-9,101,218"


### TOL

In [15]:
''' FTTx SubBase '''

v_metric_list = ['TB3S020604'] #FTTx SubBase

fttx_subbase_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_subbase_df = fttx_subbase_df.loc[fttx_subbase_df['TM_KEY_MTH']==v_tm_key_mth]
fttx_subbase_df = fttx_subbase_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
fttx_subbase_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TB3S020604,FTTx SubBase,2025-06-04 02:32:37,20250531,0,"3,294,820","3,294,739","3,294,739","3,294,739","-3,294,820"


In [16]:
''' FTTx Reported SubBase '''

v_metric_list = ['TB3S000600'] #FTTx Reported SubBase

fttx_report_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
fttx_report_sub_df = fttx_report_sub_df.loc[fttx_report_sub_df['TM_KEY_MTH']==v_tm_key_mth]
fttx_report_sub_df = fttx_report_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
fttx_report_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,TOL,TB3S000600,FTTx Reported SubBase,2025-06-04 02:32:37,20250531,0,"3,203,127","3,203,048","3,203,048","3,203,048","-3,203,127"


### TVS

In [17]:
''' TVS Active Subs '''

v_metric_list = ['TB4S000500'] #TVS Active Subs

tvs_active_sub_df = monthly_df_display.loc[monthly_df_display['METRIC_CD'].isin(v_metric_list)]
tvs_active_sub_df = tvs_active_sub_df.loc[tvs_active_sub_df['TM_KEY_MTH']==v_tm_key_mth]
tvs_active_sub_df = tvs_active_sub_df.sort_values(by='METRIC_NAME').reset_index(drop=True)
tvs_active_sub_df

,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,PPN_TM,ACTUAL_AS_OF,C,P,G,H,HH,DIFF (C-P)
0,202505,TVS,TB4S000500,TVS Active Subs,2025-06-04 02:32:37,20250531,"1,151,558","1,129,379","1,129,379","1,129,379","1,129,379","22,179"
